# HW 1- Logistic Regression
- 수업시간에 다룬 Logistic Regression Notebook을 참고로 하여 Diabetes데이터를 가지고 Logistic Regression으로 학습하고 테스트하는 모델을 작성
- 이 노트북 파일을 받아서 작성하고 화일명은 Hw1_학번 (예: Hw1_2020_12345, 반드시 하이픈대신 언더라인으로)으로 해서 etl에 탑재
- 아래의 지시사항대로 하나 필요할 경우 cell을 더 첨가해서 자유롭게 할 수 있다.
- 반드시 colab에서 실행되는지 확인 할 것.
- Due: 4월 2일 수업시간전까지

# Diabetes 데이터 셋의 구조(diabetes.csv)
![%E1%84%89%E1%85%B3%E1%84%8F%E1%85%B3%E1%84%85%E1%85%B5%E1%86%AB%E1%84%89%E1%85%A3%E1%86%BA%202020-03-30%20%E1%84%8B%E1%85%A9%E1%84%8C%E1%85%A5%E1%86%AB%2011.15.10.png](attachment:%E1%84%89%E1%85%B3%E1%84%8F%E1%85%B3%E1%84%85%E1%85%B5%E1%86%AB%E1%84%89%E1%85%A3%E1%86%BA%202020-03-30%20%E1%84%8B%E1%85%A9%E1%84%8C%E1%85%A5%E1%86%AB%2011.15.10.png)

- 8개의 자질과 맨 마지막 칼럼은 당뇨인지 아닌지, 0, 1로 되어 있음
- 총 768개의 행으로 되어 있음
- 읽을때 650개를 training 데이터로, 나머지를 test 데이터로 할 것

In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import pandas as pd


## Step 1: Loading Diabetes Dataset
- 데이터 파일 읽어와서 train_x, train_y(정답), test_x, test_y(정답)로 구분함

In [5]:
diabetesDF = pd.read_csv('diabetes.csv')

train_x = torch.Tensor(diabetesDF.drop('Outcome',1)[:650].astype(float).values)
train_y = torch.Tensor(diabetesDF['Outcome'][:650].astype(float).values)
test_x = torch.Tensor(diabetesDF.drop('Outcome',1)[650:].astype(float).values)
test_y = torch.Tensor(diabetesDF['Outcome'][650:].astype(float).values)

In [6]:
# 정답은 0, 1로 되어 있어 float -> int로 
train_y = train_y.type(torch.int64)
test_y = test_y.type(torch.int64)

## STEP 2: MAKING DATASET ITERABLE

In [7]:
import torch.utils.data

In [8]:
# train_x, train_y를 텐서데이터 셋으로 변환

train_dataset = torch.utils.data.TensorDataset(train_x, train_y)
test_dataset = torch.utils.data.TensorDataset(test_x, test_y)

In [11]:
train_y[0]

tensor(1)

In [0]:
# Hyper-parameters 

batch_size = 10
n_iters = 300
num_epochs = n_iters / (len(train_dataset) / batch_size)
num_epochs = int(num_epochs)


In [0]:
# Data loader (input pipeline)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size=batch_size, 
                                           shuffle=True)

In [0]:
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, 
                                           batch_size=batch_size, 
                                           shuffle=False)

In [10]:
import collections
isinstance(test_loader, collections.Iterable)

True

## STEP 3: CREATE MODEL CLASS

In [0]:
class LogisticRegressionModel(nn.Module):      #you have to define class
    def __init__(self, input_dim, output_dim):
        super(LogisticRegressionModel, self).__init__()
        self.linear = nn.Linear(input_dim, output_dim)
    def forward(self, x):
        out = self.linear(x)
        return out  

## STEP 4: INSTANTIATE MODEL CLASS

In [0]:
input_dim = 8  #You have to define based on the diabetes data
output_dim = 2 #You have to define based on the diabetes data


In [2]:
len(train_dataset[1][0])

NameError: name 'test_dataset' is not defined

In [0]:
# Logistic regression model
model = LogisticRegressionModel(input_dim, output_dim)
#you have to define

#######################
###  USE GPU FOR MODEL  ##
#######################

In [15]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu") ##GPU
model.to(device)   ##GPU

LogisticRegressionModel(
  (linear): Linear(in_features=8, out_features=2, bias=True)
)

## STEP 5: INSTANTIATE LOSS CLASS

In [0]:
criterion = nn.CrossEntropyLoss()  

## STEP 6: INSTANTIATE OPTIMIZER CLASS

In [0]:
learning_rate = 0.001

optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)   # you have to define


## STEP 7: TRAIN THE MODEL

In [18]:
iter = 0
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.view(-1, 8).requires_grad_().to(device)  ##GPU
        labels = labels.to(device)  ##GPU
      
        optimizer.zero_grad()
      
        outputs = model(images)
      
        loss = criterion(outputs, labels)
        
        loss.backward()
        
        optimizer.step()
        
        iter += 1
        
        if iter % 10 == 0:
            correct = 0
            total = 0
            for images, labels in test_loader:
                images = images.view(-1, 8).to(device)   ##GPU
                
                outputs = model(images)
                
                _, predicted = torch.max(outputs.data, 1)
                
                total += labels.size(0)
                
                if torch.cuda.is_available():
                    correct += (predicted.cpu() == labels.cpu()).sum() #gpu
                else:
                    correct += (predicted == labels).sum()
            
            accuracy = 100 * correct.item() / total
            
            print('Iteration: {}. Loss: {}. Accuracy: {}'.format(iter, loss.item(), accuracy))

Iteration: 10. Loss: 5.2806243896484375. Accuracy: 45.76271186440678
Iteration: 20. Loss: 0.23060913383960724. Accuracy: 54.23728813559322
Iteration: 30. Loss: 5.313769340515137. Accuracy: 61.016949152542374
Iteration: 40. Loss: 7.325562953948975. Accuracy: 44.067796610169495
Iteration: 50. Loss: 3.047623872756958. Accuracy: 62.71186440677966
Iteration: 60. Loss: 3.764162063598633. Accuracy: 65.2542372881356
Iteration: 70. Loss: 7.805091857910156. Accuracy: 61.86440677966102
Iteration: 80. Loss: 4.9595537185668945. Accuracy: 62.71186440677966
Iteration: 90. Loss: 14.393163681030273. Accuracy: 39.83050847457627
Iteration: 100. Loss: 3.852177381515503. Accuracy: 62.71186440677966
Iteration: 110. Loss: 6.071717262268066. Accuracy: 46.610169491525426
Iteration: 120. Loss: 4.840475082397461. Accuracy: 62.71186440677966
Iteration: 130. Loss: 17.336349487304688. Accuracy: 61.86440677966102
Iteration: 140. Loss: 4.39025354385376. Accuracy: 45.76271186440678
Iteration: 150. Loss: 0.726472854614

In [0]:
# Save the model checkpoint
torch.save(model.state_dict(), 'model.ckpt')